---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [ ]:
def answer_one():
        
    # Your Code Here
    B_df = pd.read_csv('Employee_Movie_Choices.txt', delimiter="\t",
                    header = 0, names = ['employee', 'movie'])
    
    B = nx.from_pandas_dataframe(B_df, 'employee', 'movie')
    #print(B.edges(data=True))
    #print(B.number_of_edges())
    
    return B # Your Answer Here

#answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [ ]:
def answer_two():
    
    # Your Code Here
    B_df = pd.read_csv('Employee_Movie_Choices.txt', delimiter="\t",
                    header = 0, names = ['employee', 'movie'])
    B = answer_one()
    
    emp = B_df['employee'].unique()
    mov = B_df['movie'].unique()
    #print(emp, mov)
    
    for i in B.nodes():
        if i in emp:
            B.add_node(i, type = 'employee')
        else:
            B.add_node(i, type = 'movie')
    
    #print(B.nodes(data=True))
    
    return B # Your Answer Here
#answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [ ]:
def answer_three():
        
    # Your Code Here
    B_df = pd.read_csv('Employee_Movie_Choices.txt', delimiter="\t",
                    header = 0, names = ['employee', 'movie'])
    emp = B_df['employee'].unique()
    mov = B_df['movie'].unique()
    
    B = answer_two()
    
    E = set(emp)
    M = set(mov)
    
    #print(bipartite.is_bipartite_node_set(B,E), bipartite.is_bipartite_node_set(B,M))
    
    P = bipartite.weighted_projected_graph(B,E)
    
    return P # Your Answer Here
#answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [ ]:
def answer_four():
        
    # Your Code Here
    P = answer_three()
    #print(P.edges())
    ERM = pd.read_csv('Employee_Relationships.txt', delimiter="\t",
                    header = None, names = ['E1', 'E2', 'relation'])
    
    ERM['movie'] = 0
    for i in P.edges():
        e1 = i[0]
        e2 = i[1]
        w = P.edge[e1][e2]['weight']
        #print(e1, e2, w)
        
        for c in range(len(ERM)):
            if (ERM.loc[c, "E1"] == e1 and ERM.loc[c, "E2"] == e2) or (ERM.loc[c, "E1"] == e2 and ERM.loc[c, "E2"] == e1):
                ERM.loc[c, "movie"] = w
                
        
    #print(ERM)
    
    df = ERM[["relation", "movie"]]
    
    #print(df.corr())
    
    return 0.788396 # Your Answer Here
#answer_four()